In [1]:
import pandas as pd 

In [6]:
df1=pd.read_csv('raw/print_monograph_hathitrust_20200504-1.txt', sep='\t')


In [5]:
df2=pd.read_csv('raw/print_monograph_hathitrust_20200504-2.txt', sep='\t')
df3=pd.read_csv('raw/print_monograph_hathitrust_20200504-3.txt', sep='\t')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
df1=df1[['35','RECORD #(BIBLIO)','TITLE','BCODE3','VOLUME', 'ISN','GOVT DOC #']]

In [16]:
df1.head()


,35,RECORD #(BIBLIO),TITLE,BCODE3,VOLUME,ISN,GOVT DOC #
0,NaN,b1000001x,Great conversation : the substance of a libera...,-,NaN,NaN,NaN
1,"0179084,""(WaOLN)rye0000002""",b10000021,Hippocratic writings / [Translated by Francis ...,-,NaN,NaN,NaN
2,"0179090,""(WaOLN)rye0000004""",b10000045,Lucretius : On the nature of things / translat...,-,NaN,NaN,NaN
3,NaN,b10000057,Poems / translated into English verse by James...,-,NaN,NaN,NaN
4,"0179095,""(WaOLN)rye0000006""",b10000069,Lives of the noble Grecians and Romans / The D...,-,NaN,NaN,NaN


In [17]:
df2=df2[['35','RECORD #(BIBLIO)','TITLE','BCODE3','VOLUME', 'ISN','GOVT DOC #']]

In [18]:
df2.head()

,35,RECORD #(BIBLIO),TITLE,BCODE3,VOLUME,ISN,GOVT DOC #
0,NaN,b1000001x,Great conversation : the substance of a libera...,-,NaN,NaN,NaN
1,"0179084,""(WaOLN)rye0000002""",b10000021,Hippocratic writings / [Translated by Francis ...,-,NaN,NaN,NaN
2,"0179090,""(WaOLN)rye0000004""",b10000045,Lucretius : On the nature of things / translat...,-,NaN,NaN,NaN
3,NaN,b10000057,Poems / translated into English verse by James...,-,NaN,NaN,NaN
4,"0179095,""(WaOLN)rye0000006""",b10000069,Lives of the noble Grecians and Romans / The D...,-,NaN,NaN,NaN


In [19]:
df3=df3[['35','RECORD #(BIBLIO)','TITLE','BCODE3','VOLUME', 'ISN','GOVT DOC #']]

In [20]:
df3.head()

,35,RECORD #(BIBLIO),TITLE,BCODE3,VOLUME,ISN,GOVT DOC #
0,NaN,b20440066,Cyber-archaeology / edited by Maurizio Forte.,-,NaN,"9781407307213 (pbk.),""1407307215 (pbk.)""",NaN
1,NaN,b20440078,After modernity : archaeological approaches to...,-,NaN,"9780199548088 (pbk.),""0199548080 (pbk.)""",NaN
2,NaN,b2044008x,Framing public memory / edited by Kendall R. P...,-,NaN,"0817313893 (cloth : alk. paper),""9780817354909""",NaN
3,NaN,b20440091,In the house of the hangman : the agonies of G...,-,NaN,"9780226626383 (alk. paper),""0226626385 (alk. p...",NaN
4,NaN,b20440108,Sex and the citizen : interrogating the Caribb...,-,NaN,"9780813931135 (pbk. : alk. paper),""0813931134 ...",NaN


In [315]:
frames = [df1, df2, df3]

mono = pd.concat(frames)

In [372]:
mono.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468932 entries, 0 to 68931
Data columns (total 7 columns):
35                  468932 non-null object
RECORD #(BIBLIO)    468932 non-null object
TITLE               468926 non-null object
BCODE3              468932 non-null object
VOLUME              17545 non-null object
ISN                 468932 non-null object
GOVT DOC #          30739 non-null object
dtypes: object(7)
memory usage: 28.6+ MB


In [305]:
mono.sample(20)

,35,RECORD #(BIBLIO),TITLE,BCODE3,VOLUME,ISN,GOVT DOC #
20574,"ocn805047072,""(OCoLC)805047072""",b24340364,Sailing to Babylon : Poems / by James Pollock ...,-,NaN,"9780986533877 (paperback),""0986533874 (paperba...",NaN
120031,"0469999,""(WaOLN)rye0167854""",b11628121,Interpreting cardiac arrhythmias : a basic guide.,-,NaN,826120105,NaN
51074,"0393373,""(WaOLN)rye0070174""",b10684049,Social problems : a critical analysis of theor...,-,NaN,669853321,NaN
160477,"93030162,""(WaOLN)rye0226498""",b12216185,Panoramas of the Far East / photographs by Loi...,-,NaN,1560983310 (pbk. : alk. paper) $15.95,NaN
86618,"0234140,""(WaOLN)rye0122736""",b11184760,Hotel Paradiso / by Georges Feydeau and Mauric...,-,NaN,435203002,NaN
90814,(WaOLN)rye0127760,b11234787,Song to David : and other poems / Selected wi...,-,NaN,NaN,NaN
172653,NaN,b12427524,Far south development area / Department of Dev...,-,NaN,NaN,NaN
120614,"0540700,""(WaOLN)rye0169157""",b11640881,Menopause / Robert B. Greenblatt John Studd ...,-,NaN,NaN,NaN
92487,(WaOLN)rye0129678,b11253794,Sir Walter Scott and his world.,-,NaN,500130329,NaN
169483,"ocm06015336,""(OCoLC)6015336""",b12384306,Problems and responses resource community dev...,-,NaN,NaN,NaN


In [25]:
import re

In [282]:
test ="9780345811820 (software),0345811828, $128.95 New York"
print(re.sub(r'\([a-z\s.]*\)','',test,flags=re.IGNORECASE))
print(re.sub(r'\$[0-9]*\.[0-9]*','',test))

9780345811820 ,0345811828, $128.95 New York
9780345811820 (software),0345811828,  New York


In [306]:
# almost there, but still not quite 

def ISSN(i):
        re.sub('\$[0-9]*\.[0-9]*','',i)
#remove price ($100.00)
        re.sub('\$[0-9]*\.[0-9]*','',i)
    #remove (softcover) (pbk.) (New York)
        re.sub('\([a-z\s.]*\)','',i,flags=re.IGNORECASE)
    #remove pbk.
        re.sub('[a-z\s.]*','',i,flags=re.IGNORECASE)
#remove all the text e.g. notation that it is softcover, hardcover
    #re.sub('[\s()\.:a-z]*','',i,flags=re.IGNORECASE)
    #re.sub('\s\([a-z]*\.*\)','',i,flags=re.IGNORECASE)
    #re.sub('\([A-Za-z]*\s[A-Za-z]*\)','',i,flags=re.IGNORECASE)
        re.sub('\(\([a-z]*\s[a-z]*\.\)\)','',i,flags=re.IGNORECASE)
#remove unwanted commas, 
        re.sub(r',,','',i,flags=re.IGNORECASE)


In [308]:
mono['ISN']=mono.ISN.astype('str')
mono['ISN']=mono.ISN.apply(ISSN)


In [378]:
mono.sample(10)

,35,RECORD #(BIBLIO),TITLE,BCODE3,VOLUME,ISN,GOVT DOC #
1858,nan,b20620408,Founding brothers : the revolutionary generati...,-,NaN,"9780375405440,""0375405445""",NaN
20761,"0737367 1981 2719048,""0700730 1979 2537783"",""0...",b10253609,Annual review.,-,NaN,,NaN
116933,0600973,b11573442,Issue paper / Local Board of Health City of T...,-,NaN,,NaN
10987,0401786,b10142605,Rise of Rome to 264 B.C.,-,NaN,0520017951,NaN
87213,0367002,b11191399,The liar.,-,NaN,,435210130
76499,0152596,b11041857,Georges Rouault / including a catalogue of wor...,-,NaN,,NaN
153547,29279912,b12088699,Desktop publishing in black + white & color : ...,-,NaN,879847000,NaN
49748,1020766,b10660331,Money in crisis : a practical solution for pay...,-,NaN,0969571208,NaN
51175,0598338,b10685637,Guide for local evaluation.,-,NaN,,NaN
40015,0633830,b10533254,User response to pedestrianised shopping streets.,-,NaN,704403633,NaN


In [379]:
def clean035(q):
    #remove WaOLN"(WaOLN)rye0053656"
    re.sub('\"\(WaOLN\)rye[01-9]+\"','',q)
    return q

In [380]:
mono['35']=mono['35'].astype('str')
mono['35']=mono['35'].apply(clean035)

In [381]:
test='0388251,"(WaOLN)rye0019428"'
test=re.sub('\"\(WaOLN\)rye[01-9]+\"','',test)
print(test)
#print(re.sub(r'\z,\z','',test))

0388251,


In [382]:
mono['ISN']=mono.ISN.astype('str')


In [383]:
mono.ISN=mono.ISN.apply(lambda x: re.sub('\$[0-9]*\.[0-9]*','',x))
mono.ISN=mono.ISN.apply(lambda x: re.sub('\([a-z\s.]*\)','',x,flags=re.IGNORECASE))
mono.ISN=mono.ISN.apply(lambda x: re.sub('[a-z\s.]*','',x,flags=re.IGNORECASE))
mono.ISN=mono.ISN.apply(lambda x: re.sub('\([:]\)','',x))

In [384]:
mono['35']=mono['35'].astype('str')
mono['35']=mono['35'].apply(lambda x:re.sub('\"\(WaOLN\)rye[01-9]+\"','',x))

In [361]:
mono['35']=mono['35'].apply(lambda x:re.sub(',\Z','',x))

In [366]:
mono.sample(10)

,35,RECORD #(BIBLIO),TITLE,BCODE3,VOLUME,ISN,GOVT DOC #
5352,nan,b10076700,Islam and the West : a historical cultural sur...,-,NaN,,NaN
69293,0440488,b10951192,Education of Black folk : the Afro-American st...,-,NaN,,0060102225 $6.95
104005,(WaOLN)rye0142004,b11376090,L'homme rapaillé.,-,NaN,"2760601471,""0840501471""",NaN
85955,0814703,b11177093,Voltaire and sensibility / R.S. Ridgway.,x,NaN,0773501304,NaN
101559,0266110,b11350817,Volvox : poetry from the unofficial languages ...,-,NaN,,NaN
48984,0535764,b1064443x,Investments.,-,NaN,013504605,NaN
102521,0481217,b11360938,Fire-eaters.,-,NaN,"0887501818,""0887501796""",NaN
187581,CANMARC/ABR-0727/ACQ2,b12681398,The living soil : land use and society : a soc...,-,NaN,919879543,NaN
79075,nan,b11078637,The anger of Achilles : Homer's Iliad / transl...,-,NaN,,NaN
66721,1001611,b10892965,Brief to the Ontario Government on access orde...,-,NaN,,NaN


In [369]:
mono.ISN.isnull().sum()

0